In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy import sparse
train_merged=pd.read_csv('train_merged.csv')
df=train_merged
v = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)
cur=df.groupby("user_id")["creative_id"].apply(list)
# cur.astype(int)
display(cur)
v_fit = v.fit_transform(cur)
display(v_fit)
sparse.save_npz('v_fit.npz', v_fit)

user_id
1         [71691, 90171, 122032, 209778, 821396, 877468,...
2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
3         [66009, 72533, 392052, 593522, 599128, 661347,...
4         [31070, 39588, 72989, 215041, 574787, 589886, ...
5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                ...                        
899996    [92193, 114074, 654526, 931537, 939409, 145880...
899997    [7400, 24333, 30210, 60330, 69204, 103918, 103...
899998    [71752, 648402, 1251649, 1370659, 1779124, 230...
899999    [12838, 12838, 12838, 12838, 122834, 279342, 3...
900000    [234063, 285502, 423617, 1224107, 1405713, 180...
Name: creative_id, Length: 900000, dtype: object

<900000x2481135 sparse matrix of type '<class 'numpy.float64'>'
	with 27608868 stored elements in Compressed Sparse Row format>

In [2]:
%reset -f
import pandas as pd
from scipy import sparse
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
# display(user_info)
agg=pd.read_csv('agg.csv')
# agg.astype(int)
# display(agg)
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_train=user_info["gender"].iloc[:num_train]
y_test=user_info["gender"].iloc[num_train:]

# X_train=sparse.csr_matrix(v_fit[:num_train,:],dtype=float).tocsr()
# X_test=sparse.csr_matrix(v_fit[num_train:,:],dtype=float).tocsr()

X_train=sparse.csr_matrix(agg.iloc[:num_train])
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_train=sparse.hstack((v_fit[:num_train,:],X_train),dtype=float).tocsr()
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()

,user_id,age,gender
0,1,4,0
1,2,10,0
2,3,7,1
3,4,5,0
4,5,4,0
...,...,...,...
899995,899996,5,0
899996,899997,3,1
899997,899998,4,1
899998,899999,3,0


,click_times_mean,advertiser_id_nunique,advertiser_id_count,industry_nunique,industry_count,time_nunique,time_count,product_category_nunique,product_category_count,product_id_nunique,product_id_count,ad_id_nunique,ad_id_count
0,1.076923,12,13,9,13,10,13,3,13,6,13,12,13
1,1.022222,36,45,15,45,28,45,3,45,20,45,42,45
2,1.000000,28,30,8,30,23,30,6,30,17,30,30,30
3,1.000000,26,29,10,29,15,29,6,29,18,29,29,29
4,1.030303,30,33,18,33,26,33,4,33,7,33,33,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,1.000000,12,14,5,14,12,14,3,14,5,14,13,14
899996,1.111111,13,18,10,18,14,18,4,18,10,18,17,18
899997,1.071429,9,14,5,14,10,14,4,14,5,14,14,14
899998,1.000000,16,22,14,22,17,22,7,22,5,22,18,22


In [3]:
# !pip install lightgbm
import lightgbm as lgb
# display(X_train,y_train)
# display(X_test,y_test)
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('CountVectorizer_train_gender.bin')
# test_data=lgb.Dataset(X_test,y_test)
# test_data.save_binary('CountVectorizer_test_gender.bin')

In [1]:
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_gender.bin')
lgb_params = {'num_leaves': 2**7-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, shuffle=True,
    early_stopping_rounds=50, verbose_eval=10, show_stdv=True)

start training
[10]	cv_agg's auc: 0.6231 + 0.00142673
[20]	cv_agg's auc: 0.628205 + 0.00127108
[30]	cv_agg's auc: 0.641034 + 0.000831938
[40]	cv_agg's auc: 0.648032 + 0.000781777
[50]	cv_agg's auc: 0.656094 + 0.00104904
[60]	cv_agg's auc: 0.66377 + 0.00129256
[70]	cv_agg's auc: 0.668926 + 0.00125677
[80]	cv_agg's auc: 0.673008 + 0.00128241
[90]	cv_agg's auc: 0.676582 + 0.00132133
[100]	cv_agg's auc: 0.679915 + 0.00122345
[110]	cv_agg's auc: 0.683032 + 0.00119218
[120]	cv_agg's auc: 0.685907 + 0.00122485
[130]	cv_agg's auc: 0.688598 + 0.00119378
[140]	cv_agg's auc: 0.691029 + 0.00108103
[150]	cv_agg's auc: 0.693345 + 0.00114
[160]	cv_agg's auc: 0.695466 + 0.00114003
[170]	cv_agg's auc: 0.697398 + 0.0011217
[180]	cv_agg's auc: 0.699246 + 0.00114483
[190]	cv_agg's auc: 0.700961 + 0.00117523
[200]	cv_agg's auc: 0.702576 + 0.00114424
[210]	cv_agg's auc: 0.704087 + 0.00115133
[220]	cv_agg's auc: 0.705509 + 0.00111808
[230]	cv_agg's auc: 0.70679 + 0.00110817
[240]	cv_agg's auc: 0.708069 + 0.0

In [ ]:
# predict gender
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_gender.bin')

lgb_params = {'num_leaves': 2**7-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=200)
model.save_model("model.txt")
print("training completed!")

In [1]:
# predict gender
%reset -f
import pandas as pd
from scipy import sparse
import lightgbm as lgb
from sklearn import metrics
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
# display(user_info)
agg=pd.read_csv('agg.csv')
# display(agg)
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_test=user_info["gender"].iloc[num_train:]
# display(y_test)
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='model.txt')
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)*1
print("acc:",metrics.accuracy_score(y_test,y_pred))
print("acc:",metrics.roc_auc_score(y_test,y_pred))
# submission=pd.DataFrame(result,columns=["gender"])
# submission["user_id"]=[i for i in range(num_train+1,user_info.shape[0]+1)]
# display(submission)

start predicting
acc: 0.7273166666666666
acc: 0.5921415079214687


In [ ]:
# cv for age
%reset -f
import lightgbm as lgb
import pandas as pd
from scipy import sparse
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
agg=pd.read_csv('agg.csv')
agg.astype(int)
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_train=user_info["age"].iloc[:num_train]
y_test=user_info["age"].iloc[num_train:]
X_train=sparse.csr_matrix(agg.iloc[:num_train])
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_train=sparse.hstack((v_fit[:num_train,:],X_train),dtype=float).tocsr()
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
train_data = lgb.Dataset(X_train,y_train)

lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'regression_l2',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'mae',
              'seed':1024,
              'nthread':12,
              'lambda_l1': 0.2,
             }
# with open('lgb.txt', 'a') as f:
#     f.write(str(lgb_params))
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=1, show_stdv=True, seed=0)

start training
[1]	cv_agg's l1: 1.66908 + 0.00406433
[2]	cv_agg's l1: 1.66908 + 0.00406146
[3]	cv_agg's l1: 1.66907 + 0.00405881
[4]	cv_agg's l1: 1.66906 + 0.00405848
[5]	cv_agg's l1: 1.66905 + 0.00406362
[6]	cv_agg's l1: 1.66906 + 0.00406845
[7]	cv_agg's l1: 1.66904 + 0.00406697
[8]	cv_agg's l1: 1.66904 + 0.00406428
[9]	cv_agg's l1: 1.66903 + 0.0040562
[10]	cv_agg's l1: 1.66903 + 0.00406892
[11]	cv_agg's l1: 1.66903 + 0.00408569
[12]	cv_agg's l1: 1.66903 + 0.00408316
[13]	cv_agg's l1: 1.66903 + 0.004088
[14]	cv_agg's l1: 1.66903 + 0.00409732
[15]	cv_agg's l1: 1.66902 + 0.00409194
[16]	cv_agg's l1: 1.66903 + 0.00409498
[17]	cv_agg's l1: 1.66904 + 0.0040898
[18]	cv_agg's l1: 1.66904 + 0.00409967
[19]	cv_agg's l1: 1.66904 + 0.00409745
[20]	cv_agg's l1: 1.66903 + 0.00410074
[21]	cv_agg's l1: 1.66903 + 0.00409969
[22]	cv_agg's l1: 1.66902 + 0.0041086
[23]	cv_agg's l1: 1.66902 + 0.00410118
[24]	cv_agg's l1: 1.66902 + 0.00409802
[25]	cv_agg's l1: 1.66901 + 0.00410802
[26]	cv_agg's l1: 1.6690

In [2]:
# %%capture cap --no-stderr
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_gender.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':10,
             }
# with open('lgb.txt', 'a') as f:
#     f.write(str(lgb_params))
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=1, show_stdv=True, seed=0)
# print('best num_boost_round:', len(cv_results['auc-mean']))
# print('best cv score:', cv_results['auc-mean'])

KeyboardInterrupt: 

In [4]:
# !pip install lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=train_merged.drop(columns=["gender","user_id"])
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("age")
X_train=train
y_test=test.pop("age")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)
train_data.save_binary('age.bin')

NameError: name 'train_merged' is not defined

In [ ]:
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('age.bin')
params = {'num_leaves': 2**7-1,
          'min_data_in_leaf': 25, 
          'objective':'regression_l2',
          'max_depth': -1,
          'learning_rate': 0.1,
          'min_child_samples': 20,
          'boosting': 'gbdt',
          'feature_fraction': 0.6,
          'bagging_fraction': 0.9,
          'bagging_seed': 11,
          'metric': 'mae',
          'seed':1024,
          'lambda_l1': 0.2,
          'nthread':12,
         }
print("start training")
cv_results = lgb.cv(
    params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=10, show_stdv=True, seed=0)
print('best num_boost_round:', len(cv_results['mae-mean']))
print('best cv score:', cv_results['mae-mean'])
# bst = lgb.train(param, train_data, num_round,categorical_feature=features)
# bst.save_model('test_model.txt')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])